In [17]:
from collections import defaultdict
from itertools import cycle, islice
from ordered_set import OrderedSet
from string import Template

import utils

In [2]:
pre_template_tr = Template("here's $agent .\n<s> hi $agent !\n<s> here's a $theme and a $recipient !\n<s>")
pre_template_rt = Template("here's $agent .\n<s> hi $agent !\n<s> here's a $recipient and a $theme !\n<s>")

do_template = Template("$agent [verb] the $recipient the $theme .")
po_template = Template("$agent [verb] the $theme to the $recipient .")

In [10]:
def create_stimuli(pre, agent, recipient, theme):
    pre = pre.substitute(agent=agent, theme=theme, recipient=recipient)
    do = do_template.substitute(agent=agent, theme=theme, recipient=recipient)
    po = po_template.substitute(agent=agent, theme=theme, recipient=recipient)
    do = f"{pre} {do}"
    po = f"{pre} {po}"
    return (do, po)


def create_entry(i, order, agent, theme, recipient, theme_animacy, dative, stimulus):
    return {
        "idx": i,
        "givenness_order": order,
        "agent": agent,
        "theme": theme,
        "recipient": recipient,
        "theme_animacy": theme_animacy,
        "dative": dative,
        "stimulus": stimulus,
    }

In [11]:
animates = ['man', 'turtle', 'dog', 'bear', 'cat', 'lady', 'boy', 'girl', 'horse', 'baby', 'bunny']
inanimates = ['cookie', 'lego', 'block', 'pencil', 'cup', 'ball', 'book', 'box', 'clothes', 'bag']

agents = ["laura", "mark", "sarah", "william", "alex"]

In [12]:
arunachalam_arguments = []
for recipient in animates:
    animate_space = OrderedSet(animates) - OrderedSet([recipient])
    inanimate_space = inanimates

    for a_theme in animate_space:
        arunachalam_arguments.append((a_theme, 'animate', recipient))
    for i_theme in inanimate_space:
        arunachalam_arguments.append((i_theme, 'inanimate', recipient))

In [13]:
len(arunachalam_arguments)

220

In [14]:
arunachalam_stimuli = []
for i, (agent, args) in enumerate(zip(cycle(agents), arunachalam_arguments)):
    theme, theme_animacy, recipient = args
    do_tr, po_tr = create_stimuli(pre_template_tr, agent, recipient, theme)
    do_rt, po_rt = create_stimuli(pre_template_rt, agent, recipient, theme)

    # arunachalam_stimuli.append((i, "theme-recipient", agent, theme, recipient, theme_animacy, 'do', do_tr))
    arunachalam_stimuli.append(
        create_entry(
            i, "theme_recipient", agent, theme, recipient, theme_animacy, "do", do_tr
        )
    )
    arunachalam_stimuli.append(
        create_entry(
            i, "theme_recipient", agent, theme, recipient, theme_animacy, "pp", po_tr
        )
    )
    arunachalam_stimuli.append(
        create_entry(
            i, "recipient_theme", agent, theme, recipient, theme_animacy, "do", do_rt
        )
    )
    arunachalam_stimuli.append(
        create_entry(
            i, "recipient_theme", agent, theme, recipient, theme_animacy, "pp", po_rt
        )
    )

In [18]:
len(arunachalam_stimuli)

880

In [20]:
utils.write_jsonl(arunachalam_stimuli, "../data/experiments/arunachalam-final.jsonl")